<a href="https://colab.research.google.com/github/PedroGalvao95/RunningDataset/blob/main/Running_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto de ETL, manipulação de dados com Python - Dados de corrida
---

Objetivos:

- documentar tudo
- combinar primeiro e último nome
- criar minutos totais
- renomear divisão por gênero
- escrever estado em latitude e longitude
---

In [43]:
# Este comando instala a biblioteca 'geopy'.

!pip install geopy


In [44]:
# 'geopy' é uma biblioteca Python que facilita a tarefa de geocodificação
#Nominatim é um módulo do geocoders dentro da bibilioteca geopy, faz a conversão de endereço em localização geográfica
#GeocoderTimedOut é uma exceção para casos que o Nominatim não responder dentro de um tempo limite
#RateLimiter é uma classe do submódulo rate_limiter, para não exceder limiter de requisições ao servidor

import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from geopy.extra.rate_limiter import RateLimiter

In [45]:
#Criação do dataframe

df = pd.read_csv("ff_race_50.csv")

In [46]:
#visualização das primeiras 5 linhas da tabela (apenas para acompanhamento)
df.head()

,PLACE,FIRST,LAST,CITY,STATE,AGE,DIVISION,TIME
0,1,Daniel,Wilson,Tulsa,OK,35,M,8:23:01
1,2,Eric,Davis,Greenwood,IN,38,M,8:57:54
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,9:24:35
3,4,Ron,Hammett,Montverde,FL,53,M,9:24:36
4,5,Seth,Cain,Geneva,FL,44,M,9:42:17


In [47]:
#limpeza de dados ausentes (possível exclusão de coluna)
df2 = df.dropna(axis=1)

In [48]:
#por não haver ausência de dados, não houve exclusão de colunas
df2.head()

,PLACE,FIRST,LAST,CITY,STATE,AGE,DIVISION,TIME
0,1,Daniel,Wilson,Tulsa,OK,35,M,8:23:01
1,2,Eric,Davis,Greenwood,IN,38,M,8:57:54
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,9:24:35
3,4,Ron,Hammett,Montverde,FL,53,M,9:24:36
4,5,Seth,Cain,Geneva,FL,44,M,9:42:17


In [49]:
#junção de primeiro e último nomes em coluna única "FULL NAME"

df2["FULL NAME"] = df2["FIRST"] + " " + df2["LAST"]

In [50]:
#adição da nova coluna FULL NAME, com nome completo
df2.head()

,PLACE,FIRST,LAST,CITY,STATE,AGE,DIVISION,TIME,FULL NAME
0,1,Daniel,Wilson,Tulsa,OK,35,M,8:23:01,Daniel Wilson
1,2,Eric,Davis,Greenwood,IN,38,M,8:57:54,Eric Davis
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,9:24:35,Stewart Edwards
3,4,Ron,Hammett,Montverde,FL,53,M,9:24:36,Ron Hammett
4,5,Seth,Cain,Geneva,FL,44,M,9:42:17,Seth Cain


In [51]:
#ajuste de dados de TIME, delimitando horas, minutos e segundos para leitura do pandas

df2["TIME"] = pd.to_timedelta(df2["TIME"])

In [52]:
#transformando dados da coluna TIME em minutos totais, e inserindo numa nova coluna
df2['TOTAL MINUTES'] = df2['TIME'].dt.total_seconds() / 60

#transformando de float para int, eliminando casas decimais
df2['TOTAL MINUTES'] = df2['TOTAL MINUTES'].astype(int)

In [53]:
df2.head()

,PLACE,FIRST,LAST,CITY,STATE,AGE,DIVISION,TIME,FULL NAME,TOTAL MINUTES
0,1,Daniel,Wilson,Tulsa,OK,35,M,0 days 08:23:01,Daniel Wilson,503
1,2,Eric,Davis,Greenwood,IN,38,M,0 days 08:57:54,Eric Davis,537
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,0 days 09:24:35,Stewart Edwards,564
3,4,Ron,Hammett,Montverde,FL,53,M,0 days 09:24:36,Ron Hammett,564
4,5,Seth,Cain,Geneva,FL,44,M,0 days 09:42:17,Seth Cain,582


In [58]:
#renomear coluna DIVISION para GENDER
df2=df2.rename(columns={'DIVISION': 'GENDER'})

In [59]:
df2.head()

,PLACE,FIRST,LAST,CITY,STATE,AGE,GENDER,TIME,FULL NAME,TOTAL MINUTES
0,1,Daniel,Wilson,Tulsa,OK,35,M,0 days 08:23:01,Daniel Wilson,503
1,2,Eric,Davis,Greenwood,IN,38,M,0 days 08:57:54,Eric Davis,537
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,0 days 09:24:35,Stewart Edwards,564
3,4,Ron,Hammett,Montverde,FL,53,M,0 days 09:24:36,Ron Hammett,564
4,5,Seth,Cain,Geneva,FL,44,M,0 days 09:42:17,Seth Cain,582


In [60]:
#função de detectar latitude e longitude
def get_lat_long(city, state):
  address = f"{city}, {state}"
  try:
    geolocator = Nominatim(user_agent="running", timeout=10)
    location = geolocator.geocode(address)
    if location:
      return location.latitude, location.longitude
    else:
      return None, None
  except GeocoderTimedOut:
    return None, None


In [61]:
#criando latitude e longitude
df2["LATITUDE"], df2["LONGITUDE"] = zip(*df2.apply(lambda x: get_lat_long(x["CITY"], x["STATE"]), axis=1))

In [62]:
df2.head()

,PLACE,FIRST,LAST,CITY,STATE,AGE,GENDER,TIME,FULL NAME,TOTAL MINUTES,LATITUDE,LONGITUDE
0,1,Daniel,Wilson,Tulsa,OK,35,M,0 days 08:23:01,Daniel Wilson,503,36.156312,-95.992752
1,2,Eric,Davis,Greenwood,IN,38,M,0 days 08:57:54,Eric Davis,537,39.613699,-86.109543
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,0 days 09:24:35,Stewart Edwards,564,29.025813,-80.927127
3,4,Ron,Hammett,Montverde,FL,53,M,0 days 09:24:36,Ron Hammett,564,28.600277,-81.673964
4,5,Seth,Cain,Geneva,FL,44,M,0 days 09:42:17,Seth Cain,582,28.737294,-81.113421


In [70]:
#juntando latitude e longitude numa coluna
df2["LAT AND LONG"] = df2["LATITUDE"].astype(str) + ", " + df2["LONGITUDE"].astype(str)

In [71]:
df2.head()

,PLACE,FIRST,LAST,CITY,STATE,AGE,GENDER,TIME,FULL NAME,TOTAL MINUTES,LATITUDE,LONGITUDE,LAT AND LONG
0,1,Daniel,Wilson,Tulsa,OK,35,M,0 days 08:23:01,Daniel Wilson,503,36.156312,-95.992752,"36.1563122, -95.9927516"
1,2,Eric,Davis,Greenwood,IN,38,M,0 days 08:57:54,Eric Davis,537,39.613699,-86.109543,"39.6136987, -86.1095429"
2,3,Stewart,Edwards,New Smyrna Beach,FL,43,M,0 days 09:24:35,Stewart Edwards,564,29.025813,-80.927127,"29.0258132, -80.9271271"
3,4,Ron,Hammett,Montverde,FL,53,M,0 days 09:24:36,Ron Hammett,564,28.600277,-81.673964,"28.6002769, -81.673964"
4,5,Seth,Cain,Geneva,FL,44,M,0 days 09:42:17,Seth Cain,582,28.737294,-81.113421,"28.737294, -81.1134205"


In [72]:
#transformando df2 em doc .csv
df2.to_csv("racingcleanupdated.csv", index=False)